# Student Record File Workflow

## Overview
Functions to obtain and clean data required for the Student Record File in Python using Census API and NCES files. 

The workflow produces a Student Person Record (SREC) file that can be linked to the Person Record File. The file includes student level records with sex, grade level, race and ethnicity.

Based on NCES data. 

The output of this workflow is a CSV file with the student record file.

The output CSV is designed to be used in the Interdependent Networked Community Resilience Modeling Environment (IN-CORE) for the housing unit allocation model.

IN-CORE is an open source python package that can be used to model the resilience of a community. To download IN-CORE, see:

https://incore.ncsa.illinois.edu/


## Instructions
Users can run the workflow by executing each block of code in the notebook.

Users can modify the code to select one county or multiple counties.

## Description of Program
- program:    ncoda_07ev1_run_SREC_workflow
- task:       Obtain School Location and Attendance Boundaries
- Version:    2023-02-10
- project:    Interdependent Networked Community Resilience Modeling Environment (IN-CORE) Subtask 5.2 - Social Institutions
- funding:	  NIST Financial Assistance Award Numbers: 70NANB15H044 and 70NANB20H008 
- author:     Nathanael Rosenheim

- Suggested Citation:
Rosenheim, Nathanael (2021) “Detailed Household and Housing Unit Characteristics: Data and Replication Code.” DesignSafe-CI. 
https://doi.org/10.17603/ds2-jwf6-s535.

## Setup Python Environment

In [1]:
# Import Python Packages Required for program
import pandas as pd       # Pandas for reading in data 
import geopandas as gpd   # Geopandas for reading Shapefiles
import numpy as np        # Numpy for working with arrays
import os                 # Operating System (os) For folders and finding working directory
import sys
import zipfile            # Zipfile for working with compressed Zipped files
import wget               # Wget for downloading files from the web
import scooby # Reports Python environment

In [2]:
# Generate report of Python environment
print(scooby.Report(additional=['pandas']))


--------------------------------------------------------------------------------
  Date: Tue Feb 14 09:37:42 2023 Central Standard Time

                OS : Windows
            CPU(s) : 12
           Machine : AMD64
      Architecture : 64bit
               RAM : 31.6 GiB
       Environment : Jupyter

  Python 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:14:58) [MSC
  v.1929 64 bit (AMD64)]

            pandas : 1.3.5
             numpy : 1.24.2
             scipy : 1.10.0
           IPython : 8.10.0
        matplotlib : 3.6.3
            scooby : 0.5.12
--------------------------------------------------------------------------------


In [3]:
#To replicate this notebook Clone the Github Package to a folder that is a sibling of this notebook.
# To access the sibling package you will need to append the parent directory ('..') to the system path list.
# append the path of the directory that includes the github repository.
# This step is not required when the package is in a folder below the notebook file.
github_code_path  = ""
sys.path.append(github_code_path)

In [4]:
os.getcwd()

'c:\\Users\\nathanael99\\MyProjects\\github\\intersect-community-data'

In [5]:
# To reload submodules need to use this magic command to set autoreload on
%load_ext autoreload
%autoreload 2
# open, read, and execute python program with reusable commands
from pyncoda.CommunitySourceData.nces_ed_gov.nces_01a_downloadfiles \
    import *
from pyncoda.CommunitySourceData.nces_ed_gov.nces_00c_cleanutils \
    import *
from pyncoda.CommunitySourceData.nces_ed_gov.nces_02c_SRECcleanCCD \
    import *
from pyncoda.CommunitySourceData.nces_ed_gov.nces_02d_SRECtidy \
    import *

## Obtain NCES Files
This section of code provides details on the web addresses for obtaining the NCES data. These datafiles are quiet large. It is recommended that the files are downloaded once. To facilitate the downloading of the files a Comma Seperated Values (CSV) file was create using Microsoft Excel (note CSV files are easier to read into the notebook). The CSV file includes the descriptions and important file names to be obtained. This input file can be modified for different school years.

In [26]:
folder_path = 'pyncoda\\CommunitySourceData\\nces_ed_gov\\'
filename = 'nces_00b_ObtainSchoolData_2023-02-10.csv'
downloadlistcsv = folder_path + filename
#county_list = ['37155']
#communityname = "RobesonCounty_NC"
county_list = ['37155']
communityname = "RobesonCounty_NC"
outputfolder = f"OutputData\\{communityname}\\01_CommunitySourceData"
outputfolder_tidy = f"OutputData\\{communityname}\\02_TidyCommunitySourceData"

year = '2015-2016'

In [ ]:
# create output folders for hui data generation
outputfolders = directory_design(state_county_name = state_county_name,
                                    outputfolder = self.outputfolder)
                    

In [7]:
schoollist_community = create_schoolist_community(downloadlistcsv, 
                        county_list,
                        communityname,
                        outputfolder,
                        year)

File EDGES_GEOCODED_SCHOOLDATA_2015-2016_RobesonCounty_NC.shp already exists.
File will not be overwritten.


In [10]:
SAB_community = create_sab_community(downloadlistcsv, 
                        schoollist_community,
                        communityname,
                        outputfolder,
                        year)

File EDGES_SAB_2015-2016_RobesonCounty_NC.shp already exists.
File will not be overwritten.


In [14]:
sab_boundaries = split_SAB_gradelevel(SAB_community,
                    outputfolder,
                    year)

PrimarySAB
MiddleSAB
HighSAB
OtherSAB
    No data for  OtherSAB . Skipping.
OpenEnrollSAB


## Generate NCES Student Record File
Based on Common Core Data file for schools.

In [35]:
ccd_df = nces_clean_student_ccd(outputfolder)

File OutputData\RobesonCounty_NC\01_CommunitySourceData/nces_ccd_RobesonCounty_NC_09.csv Already exists - Skipping Clean CCD NCES.


In [36]:
ccd_df.head()

,NCESSCH,FIPST,LEAID,SCHNO,STID09,SEASCH09,LEANM09,SCHNAM09,PHONE09,MSTREE09,...,HPALF09,TR09,TRALM09,TRALF09,TOTETH09,TotalStudents,CheckTotal1,CheckTotal2,CountFlag1,CountFlag2
0,370004002349,37,3700040,2349,78A,0,CIS ACADEMY,CIS ACADEMY,9105211669,PO BOX 706,...,0,0,0,0,109,109,0,0,0,0
1,370393001569,37,3703930,1569,780,320,ROBESON COUNTY SCHOOLS,DEEP BRANCH ELEMENTARY,9107382514,4045 DEEP BRANCH ROAD,...,0,0,0,0,477,477,0,0,0,0
2,370393001570,37,3703930,1570,780,324,ROBESON COUNTY SCHOOLS,FAIRGROVE MIDDLE,9106288290,1953 FAIRGROVE SCH ROAD,...,0,0,0,0,318,318,0,0,0,0
3,370393001571,37,3703930,1571,780,328,ROBESON COUNTY SCHOOLS,GREEN GROVE ELEMENTARY,9106287433,1850 SCHOOL ROAD,...,0,0,0,0,252,252,0,0,0,0
4,370393001572,37,3703930,1572,780,336,ROBESON COUNTY SCHOOLS,LITTLEFIELD MIDDLE,9106716065,9674 NC HWY 41 N,...,0,0,0,0,781,781,0,0,0,0


In [40]:
srec_df = tidy_SREC_nces(outputfolder_tidy, 
                ccd_df = ccd_df,
                communityname = communityname,
                year = '09')


******************************
 Reshaping data from wide to long
******************************
******************************
 Add characteristic variables
******************************
      Add race =  American Indian/Alaska Native
      Add race =  Asian/Hawaiian Native/Pacific Islander
      Add race =  Hispanic
      Add race =  Black, non-Hispanic
      Add race =  White, non-Hispanic
******************************
 Expand Dataset
******************************
     Add Counter
     Generate unique ID
     Counter max length 4


In [41]:
# Manually fix issue with L Gilbert Carroll Middle SAB
condition1 = (srec_df['NCESSCH'] == '370393002235')
conditions = condition1
# Gilbert Carroll is an overlapping SAB
srec_df.loc[conditions,'ncessch_5'] = '370393002235'

# Manually fix issue with CIS ACADEMY
# Located in Pembroke, North Carolina
condition1 = (srec_df['NCESSCH'] == '370004002349')
conditions = condition1
srec_df.loc[conditions,'ncessch_6'] = '370004002349'

In [42]:
srec_df.head()


,index,NCESSCH,SCHNAM09,LEVEL09,CHARTR09,LATCOD09,LONCOD09,racecat5,race,hispan,...,srecid,ncessch_1,ncessch_2,ncessch_3,ncessch_4,gradelevel1,gradelevel2,gradelevel3,ncessch_5,ncessch_6
0,1,370393001569,DEEP BRANCH ELEMENTARY,1,2,34.623182,-79.122924,1.0,3.0,0.0,...,S370393001569syr20092010c0001,370393001569,NaN,NaN,NaN,PK,PK,PK,NaN,NaN
1,1,370393001569,DEEP BRANCH ELEMENTARY,1,2,34.623182,-79.122924,1.0,3.0,0.0,...,S370393001569syr20092010c0002,370393001569,NaN,NaN,NaN,PK,PK,PK,NaN,NaN
2,1,370393001569,DEEP BRANCH ELEMENTARY,1,2,34.623182,-79.122924,1.0,3.0,0.0,...,S370393001569syr20092010c0003,370393001569,NaN,NaN,NaN,PK,PK,PK,NaN,NaN
3,1,370393001569,DEEP BRANCH ELEMENTARY,1,2,34.623182,-79.122924,1.0,3.0,0.0,...,S370393001569syr20092010c0004,370393001569,NaN,NaN,NaN,PK,PK,PK,NaN,NaN
4,1,370393001569,DEEP BRANCH ELEMENTARY,1,2,34.623182,-79.122924,1.0,3.0,0.0,...,S370393001569syr20092010c0005,370393001569,NaN,NaN,NaN,PK,PK,PK,NaN,NaN


In [46]:

srec_df.srecid.astype(str).describe()


count                             24024
unique                            24024
top       S370393001569syr20092010c0001
freq                                  1
Name: srecid, dtype: object

In [47]:
srec_df[['CHARTR09','SCHNAM09','LEVEL09','gradelevel']].\
    groupby(['CHARTR09','LEVEL09','gradelevel']).describe()

SCHNAM09                                      
                               count unique                       top freq
CHARTR09 LEVEL09 gradelevel                                               
1        2       G06              32      1               CIS ACADEMY   32
                 G07              39      1               CIS ACADEMY   39
                 G08              38      1               CIS ACADEMY   38
2        1       G01            2017     23      EAST ROBESON PRIMARY  162
                 G02            1849     23    SAINT PAULS ELEMENTARY  152
                 G03            1946     23      EAST ROBESON PRIMARY  168
                 G04            1790     21      ROSENWALD ELEMENTARY  150
                 G05             958     12       PEMBROKE ELEMENTARY  135
                 G06             571      8       PROSPECT ELEMENTARY  113
                 G07             332      3       MAGNOLIA ELEMENTARY  138
                 G08             313      3       MAGNOLIA ELEMENTARY  150
                 KG             1689     23    SAINT PAULS ELEMENTARY  144
                 PK              500     22      EAST ROBESON PRIMARY   48
         2       G04             212      2        LITTLEFIELD MIDDLE  151
                 G05             901      7  L GILBERT CARROLL MIDDLE  313
                 G06            1231     10  L GILBERT CARROLL MIDDLE  289
                 G07            1454     10     LUMBERTON JUNIOR HIGH  311
                 G08            1377     10     LUMBERTON JUNIOR HIGH  287
         3       G09            2132      7     LUMBERTON SENIOR HIGH  658
                 G10            1850      7     LUMBERTON SENIOR HIGH  572
                 G11            1474      7     LUMBERTON SENIOR HIGH  471
                 G12            1319      7     LUMBERTON SENIOR HIGH  382